In [3]:
import ezomero
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import sys
import os
import biohack_utils.omero_annotation as bhoa

import numpy as np
from omero.gateway import BlitzGateway
from omero.model import DatasetI, ImageI
from omero.rtypes import rstring, rint


# Load environment variables from .env file
load_dotenv(override = True)

from micro_sam.automatic_segmentation import get_predictor_and_segmenter, automatic_instance_segmentation
predictor, segmenter = get_predictor_and_segmenter(
    model_type="vit_b_lm",  # You can also use 'vit_b', 'vit_l', 'vit_h', 'vit_b_256' or your custom model checkpoint path.
    checkpoint=None,  # Replace this with your custom checkpoint.
    is_tiled=False,  # Switch to 'True' in case you would like to perform tiling-window based prediction.
)

In [2]:
def post_image(conn, img_array, image_name, dataset_id):
    # img_array should be 5D: (T, Z, C, Y, X)
    size_t, size_z, size_c, size_y, size_x = img_array.shape
    pixels = []
    for t in range(size_t):
        for z in range(size_z):
            for c in range(size_c):
                plane = img_array[t, z, c, :, :]
                pixels.append(plane)
    # Create image (pass an iterator, not a list)
    image = conn.createImageFromNumpySeq(
        iter(pixels),  # <-- fix here
        image_name,
        sizeZ=size_z,
        sizeC=size_c,
        sizeT=size_t,
        dataset=conn.getObject("Dataset", dataset_id),
        channelList=None,
        description="Segmentation mask"
    )
    return image.getId()


In [5]:
conn = ezomero.connect(
    os.getenv("OMERO_USERNAME"), 
    os.getenv("OMERO_PASSWORD"), 
    os.getenv("OMERO_GROUP"),
    host=os.getenv("OMERO_HOST"),
    port=int(os.getenv("OMERO_PORT")),
    secure=os.getenv("OMERO_SECURE") == "True"
)

In [4]:
dataset_id = 4272  # Replace with your dataset ID
im_ids = ezomero.get_image_ids(conn, dataset=dataset_id)

In [ ]:
# Get images from OMERO with ezomero
segmentation = []
for id in im_ids:
    img,img_array = ezomero.get_image(conn, id)
    #run micro_sam segmentation
    segmentation.append(automatic_instance_segmentation(
        predictor=predictor,
        segmenter=segmenter,
        input_path=img_array[0,0,:,:,0],
        ndim=2,
        tile_shape=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a tile shape
        halo=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a halo shape.
    ))


In [ ]:
# Upload the segmentation masks
seg_ids = []
for i, seg in enumerate(segmentation):
    # Convert 2D (Y,X) to 5D (T,Z,C,Y,X) with single planes/channels/timepoint
    seg_5d = seg[np.newaxis, np.newaxis, np.newaxis, :, :]
    seg_ids.append(post_image(conn, seg_5d, image_name=f"segmentation_{im_ids[i]}", dataset_id=dataset_id))


In [ ]:
#for every source image, create a collection for the segmentation
for i, source_id in enumerate(im_ids):

    coll_id = bhoa._create_collection(conn, "micro_sam_segmentation")
    bhoa._link_collection_to_image(conn, coll_id, source_id)
    bhoa._add_node_annotation(conn, source_id, "intensities", coll_id, node_name="source_image",
                                    attributes={"origin":"raw","description":"Original multichannel image","path":"source_image"})

    bhoa._link_collection_to_image(conn, coll_id, seg_ids[i])
    bhoa._add_node_annotation(conn, seg_ids[i], "annotations", coll_id, node_name="micro_sam_segmentation",
                                                        attributes={"origin":"masks","description":"Micro-SAM cell segmentation masks","path":"micro_sam_segmentation","source":"source_image"})

## Try to append a second segmentation to an exisiting collection

In [ ]:
# We will do another segmentation on a different channel
# Get images from OMERO with ezomero
segmentation = []
for id in im_ids:
    img,img_array = ezomero.get_image(conn, id)
    #run micro_sam segmentation
    segmentation.append(automatic_instance_segmentation(
        predictor=predictor,
        segmenter=segmenter,
        input_path=img_array[0,0,:,:,2],
        ndim=2,
        tile_shape=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a tile shape
        halo=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a halo shape.
    ))


Initialize instance segmentation with decoder: 100%|██████████| 1/1 [00:03<00:00,  3.78s/it]


In [11]:
# Upload the segmentation masks
seg_ids = []
for i, seg in enumerate(segmentation):
    # Convert 2D (Y,X) to 5D (T,Z,C,Y,X) with single planes/channels/timepoint
    seg_5d = seg[np.newaxis, np.newaxis, np.newaxis, :, :]
    seg_ids.append(post_image(conn, seg_5d, image_name=f"segmentation_{im_ids[i]}", dataset_id=dataset_id))


In [ ]:
#for every source image, create a collection for the segmentation
for i, im_id in enumerate(im_ids):

    collections = bhoa._get_collections(conn, im_id)
    coll_id = collections[0]['collection_id']
    if coll_id is None:
        coll_id = bhoa._create_collection(conn, "micro_sam_segmentation")
        bhoa._link_collection_to_image(conn, coll_id, im_id)
        bhoa._add_node_annotation(conn, im_id, "intensities", coll_id, node_name="raw")

    bhoa._link_collection_to_image(conn, coll_id, seg_ids[i])
    bhoa._add_node_annotation(conn, seg_ids[i], "annotations", coll_id, node_name="micro_sam_nuclei_segmentation",
                                                        attributes={"origin":"masks","description":"Micro-SAM cell nuclei masks","path":"micro_sam_nuclei_segmentation","source":"source_image"})


11000
11006
11003


In [28]:
# Show how a collection looks like
import pprint
coll_id = get_collections(conn, im_ids[0])
pprint.pprint(coll_id)


[{'collection_id': 11000,
  'members': [{'image_id': 35478,
               'node_info': {'collection_id': '11000',
                             'name': 'raw',
                             'type': 'intensities'}},
              {'image_id': 35485,
               'node_info': {'collection_id': '11000',
                             'name': 'micro_sam_segmentation',
                             'type': 'labels'}},
              {'image_id': 35489,
               'node_info': {'collection_id': '11000',
                             'name': 'micro_sam_segmentation',
                             'type': 'labels'}}],
  'name': 'micro_sam_segmentation',
  'version': '0.x'}]


In [31]:
find_related_images(conn, im_ids[0])    

[{'image_id': 35485,
  'collection_id': 11000,
  'node_info': {'type': 'labels',
   'collection_id': '11000',
   'name': 'micro_sam_segmentation'}},
 {'image_id': 35489,
  'collection_id': 11000,
  'node_info': {'type': 'labels',
   'collection_id': '11000',
   'name': 'micro_sam_segmentation'}}]

# Useful stuff

In [16]:
#delete annotations
map_ann_to_delete = ezomero.get_map_annotation_ids(conn, object_type="Image", object_id=35491)

conn.deleteObjects('Annotation', map_ann_to_delete, wait=True)  

72f8d9a8-b838-43de-872f-0c4607d2aa62/IHandle3f658974-d5e9-43c9-8c48-1761309902cd -t -e 1.1:tcp -h 128.176.233.5 -p 44685 -t 60000

In [21]:
coll_id = create_collection(conn, "image")
link_collection_to_image(conn, coll_id, 35480)
add_node_annotation(conn, 35480, "Intensities", coll_id, node_name="Raw")
link_collection_to_image(conn, coll_id, 35486)
add_node_annotation(conn, 35486, "Labels", coll_id, node_name="Nuclei_segmentation")
link_collection_to_image(conn, coll_id, 35491)
add_node_annotation(conn, 35491, "Labels", coll_id, node_name="Cell_segmentation")

11033